In [19]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
! echo "800 MHz"
! ssh htseng@eevee "cd /nfshome/htseng/courses/CSE142/demo/parallel_architectures/power_energy; sudo sh /nfshome/htseng/courses/CSE142/demo/parallel_architectures/power_energy/changeMaxFreq.sh; sudo likwid-perfctr -g ENERGY ./popcounts/popcount_SSE42"


800 MHz
--------------------------------------------------------------------------------
CPU name:	Intel(R) Xeon(R) Silver 4108 CPU @ 1.80GHz
CPU type:	Intel Skylake SP processor
CPU clock:	1.80 GHz
--------------------------------------------------------------------------------
Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
Result: 32000010811
--------------------------------------------------------------------------------
Group 1: ENERGY
+-----------------------+---------+----------+--------+--------+--------+--------+--------+--------+---------+----------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------------+---------+---------+---------+---------+---------+---------+---------+
|         Event         | Counter |  Core 0  | Core 1 | Core 2 | Core 3 | Core 4 | Core 5 | Core 6 |  Core 7

In [ ]:
! echo "3 GHz"
! ssh htseng@eevee "cd ~/courses/CSE142/demo/parallel_architectures/power_energy; sudo sh /nfshome/htseng/courses/CSE142/demo/parallel_architectures/power_energy/restoreMaxFreq.sh; sudo likwid-perfctr -g ENERGY ./popcounts/popcount_SSE42"


In [ ]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

In [ ]:
render_code("matrix_mul/blockmm.c", show=["//START_SSE","//END_SSE"])

In [ ]:
! make -C matrix_mul blockmm blockmm_SSE

In [ ]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 32 1 >> ./matrix_mul/blockmm.csv ;./matrix_mul/blockmm 1024 256 >> ./matrix_mul/blockmm.csv ; ./matrix_mul/blockmm 2048 256 >> ./matrix_mul/blockmm.csv; ./matrix_mul/blockmm 4096 256 >> ./matrix_mul/blockmm.csv

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

In [ ]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_SSE.csv
! ./matrix_mul/blockmm_SSE 32 1 >> ./matrix_mul/blockmm_SSE.csv ;./matrix_mul/blockmm_SSE 1024 256 >> ./matrix_mul/blockmm_SSE.csv ; ./matrix_mul/blockmm_SSE 2048 256 >> ./matrix_mul/blockmm_SSE.csv; ./matrix_mul/blockmm_SSE 4096 256 >> ./matrix_mul/blockmm_SSE.csv

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm_SSE.csv"))

In [ ]:
render_code("vmul/vmul_cuda.cu", show=["//START","//END"])

In [ ]:
! cd vmul; make clean; make
! echo "THREADS,CPUTIME,HOST2GPU,GPUTIME,GPU2HOST" > ./vmul/vmul.csv
! echo "CPU based vul"
! time ./vmul/vmul 33554432 1 0 30 2>> ./vmul/vmul.csv
! echo "GPU based vul"
### i stands for "How many iterations each thread performs 
### -- the larger the number, the fewer the parallelism
! for i in 1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192; do time ./vmul/vmul 33554432 $i 1 30 2>> ./vmul/vmul.csv ; done

In [ ]:
df = render_csv("./vmul/vmul.csv")
df["TOTAL"] = df["CPUTIME"] + df["HOST2GPU"] + df["GPUTIME"] + df["GPU2HOST"]
df = df.sort_values(by=["THREADS"], ascending=True)
display_df_mono(df)
plotPE(df=df, lines=True, what=[ ('THREADS', "TOTAL"), ('THREADS', "GPUTIME")], columns=2)